In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("..")
warnings.filterwarnings('ignore')

In [3]:
from scripts.data_vizualization import Data_Viz 
from scripts.data_cleaning import DataCleaner
from scripts.data_transformation import DataTransformer

DC = DataCleaner()
DV = Data_Viz()
DT = DataTransformer()

ModuleNotFoundError: No module named 'scripts.data_vizualization'

In [ ]:
# Reading the datasets

train_data = pd.read_csv('../data/train_store.csv')
test_data = pd.read_csv('../data/test_store.csv')

In [ ]:
# Finding out the total promotion percentage in both the training and test dataset

promo_test_percentage = test_data['Promo'].value_counts(0)[1]/(test_data['Promo'].value_counts(0)[1]+test_data['Promo'].value_counts(0)[0])
promo_train_percentage = train_data['Promo'].value_counts(0)[

In [ ]:
# Printing the number of percentages

print(f'There are {promo_test_percentage:.2%} of promotion in the test data set ')
print(f'There are {promo_train_percentage:.2%} of promotion in the train data set ')


In [ ]:
# Creating a test and train promo data frame for visualization

promo_test_df = test_data['Promo'].value_counts()
promo_train_df = train_data['Promo'].value_counts()
# Creating a data frame for visualization

test_df = pd.DataFrame(promo_test_df)
train_df = pd.DataFrame(promo_train_df)
# Renaming the columns for visualization

promo = [0,1]
test_df['Promotion'] = promo
train_df['Promotion'] = promo
test_df.rename(columns={'Promo':'Count'},inplace=True)
train_df.rename(columns={'Promo':'Count'},inplace=True)
# Plotting the test promo distribution 

test_df.plot(kind='bar',rot=0)
plt.xlabel('Promotion')
plt.ylabel('Count')
plt.title('Promotion distribution of test dataset')
plt.gca().get_legend().remove()
plt.savefig('../charts/test_dataset_promo_distribution.jpg')
plt.show()

In [ ]:
# Plotting the train promo distribution 

train_df.plot(kind='bar',rot=0)
plt.xlabel('Promotion')
plt.ylabel('Count')
plt.title('Promotion distribution of train dataset')
plt.gca().get_legend().remove()
plt.savefig('../charts/train_dataset_promo_distribution.jpg')
plt.show()

In [ ]:
# Finding out the state holiday field

train_data['StateHoliday'].value_counts().index.tolist()

In [ ]:
# Changing all the fields of the state holiday into string

train_data['StateHoliday'] = train_data['StateHoliday'].values.astype(str)
train_data['StateHoliday'].value_counts().index

In [ ]:
# Changing the state holiday fields to easy understandable names
 
train_data['StateHoliday'].replace({'0':'No Holiday','a':'Public Holiday','b':'Easter Holiday','c':'Christmas Holiday'},inplace=True)
# Ensuring if the train data column values has changed

train_data['StateHoliday'].value_counts().index.tolist()

In [ ]:
# Separating the date column for further analysis

DC.convert_to_datetime(train_data, ['Date'])
train_data['Year'] = train_data['Date'].apply(lambda x: x.year)
train_data['Month'] = train_data['Date'].apply(lambda x: x.month)
train_data['DayOfMonth'] = train_data['Date'].apply(lambda x: x.day)
train_data['Weekday'] = train_data['DayOfWeek'].apply(lambda x: 0 if (x in [6, 7]) else 1) # Identifying if the day is weekend or weekday
# Create a new analysed train_data

train_data.to_csv('../data/train_data_clean.csv',index=False)

In [ ]:
# Reading the data and making the index the date to make it easier for the time series analysis

train = pd.read_csv('../data/train_data_clean.csv',index_col='Date')
test = pd.read_csv('../data/train_data_clean.csv',index_col='Date')
# Finding out all the easter holidays

easter_holiday_list = train['StateHoliday'].where(train['StateHoliday']=='Easter Holiday')
easter_holiday_list = easter_holiday_list.dropna()
print(f'These are the easter holidays from 2013-2015:\n {easter_holiday_list.index.unique().tolist()}')


In [ ]:
# Finding out all the public holidays

public_holiday_list = train['StateHoliday'].where(train['StateHoliday']=='Public Holiday')
public_holiday_list = public_holiday_list.dropna()
print(f'These are the public holidays from 2013-2015:\n {public_holiday_list.index.unique().tolist()}')


In [ ]:
# Finding out all the public holidays

public_holiday_list = train['StateHoliday'].where(train['StateHoliday']=='Public Holiday')
public_holiday_list = public_holiday_list.dropna()
print(f'These are the public holidays from 2013-2015:\n {public_holiday_list.index.unique().tolist()}')


In [ ]:
# Finding out all the Christmas holidays

christmas_holiday_list = train['StateHoliday'].where(train['StateHoliday']=='Christmas Holiday')
christmas_holiday_list = christmas_holiday_list.dropna()
print(f'These are the Christmas holidays from 2013-2015:\n {christmas_holiday_list.index.unique().tolist()}')


In [ ]:
# Sales Comparision on State Holiday

train_store_holiday = train.groupby('StateHoliday').agg({'Sales': 'mean'})
train_store_holiday['StateHoliday'] = train_store_holiday.index
# Plotting the average sales per State holidays

train_store_holiday.plot(kind='bar',x='StateHoliday',y='Sales',rot='0',color=['black', 'red', 'green', 'blue'])
plt.title('Average sales')
plt.xlabel('State Holidays')
plt.ylabel('Average Sales')
plt.gca().get_legend().remove() # Removing the legend since we are working on univariate analysis
plt.savefig('../charts/average_sales_analysis.jpg')
plt.show()

In [ ]:
# Creating a date column from the index 

train['date'] = train.index
train['date'] = pd.to_datetime(train['date'])
train.info()

In [ ]:
# Creating the before after and during time stamp dataframe for 2014/15

before_index = (train["date"] >= pd.to_datetime("2014-11-25")) & (train["date"] < pd.to_datetime("2014-12-25")) # Taking the one month before xmass
before_xmass = train[before_index].groupby("date").agg({"Sales": "mean"})

during_index = (train["date"] >= pd.to_datetime("2014-12-25")) & (train["date"] < pd.to_datetime("2014-12-30")) # Thinking that the holiday will last for 6 days
during_xmass = train[during_index].groupby("date").agg({"Sales": "mean"})

after_index = (train["date"] >= pd.to_datetime("2015-01-01")) & (train["date"] < pd.to_datetime("2015-02-02"))# Taking the one month after xmass
after_xmass = train[after_index].groupby("date").agg({"Sales": "mean"})
# Plotting the graph of xmass sales before, after and during

plt.figure(figsize=(12, 6))

sns.lineplot(x = before_xmass.index, y = before_xmass["Sales"], label='Before')
sns.lineplot(x = during_xmass.index, y = during_xmass["Sales"], label='During')
sns.lineplot(x = after_xmass.index, y = after_xmass["Sales"], label='After')

plt.title("Christmass Sales 2014/2015", size=20)
plt.xticks(rotation=75, fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel(xlabel="Date", fontsize=16)
plt.ylabel(ylabel="Avg Sales", fontsize=16)
plt.savefig('../charts/christmass_sales_analysis_14-15.jpg')
plt.show()

In [ ]:
# Creating the before after and during time stamp dataframe for 2013/14

before_index = (train["date"] >= pd.to_datetime("2013-11-25")) & (train["date"] < pd.to_datetime("2013-12-25")) # Taking the one month before xmass
before_xmass = train[before_index].groupby("date").agg({"Sales": "mean"})

during_index = (train["date"] >= pd.to_datetime("2013-12-25")) & (train["date"] < pd.to_datetime("2013-12-30")) # Thinking that the holiday will last for 6 days
during_xmass = train[during_index].groupby("date").agg({"Sales": "mean"})

after_index = (train["date"] >= pd.to_datetime("2014-01-01")) & (train["date"] < pd.to_datetime("2014-02-02"))# Taking the one month after xmass
after_xmass = train[after_index].groupby("date").agg({"Sales": "mean"})
# Plotting the graph of xmass sales before, after and during

plt.figure(figsize=(12, 6))

sns.lineplot(x = before_xmass.index, y = before_xmass["Sales"], label='Before')
sns.lineplot(x = during_xmass.index, y = during_xmass["Sales"], label='During')
sns.lineplot(x = after_xmass.index, y = after_xmass["Sales"], label='After')

plt.title("Christmass Sales 2013/2014", size=20)
plt.xticks(rotation=75, fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel(xlabel="Date", fontsize=16)
plt.ylabel(ylabel="Avg Sales", fontsize=16)
plt.savefig('../charts/christmass_sales_analysis_13-14.jpg')
plt.show()

In [ ]:
easter_holiday_list.index.unique().tolist()

In [ ]:
# Creating the before after and during time stamp dataframe for 2013/14

before_index = (train["date"] >= pd.to_datetime("2013-02-28")) & (train["date"] < pd.to_datetime("2013-03-29")) # Taking the one month before easter
before_easter = train[before_index].groupby("date").agg({"Sales": "mean"})

during_index = (train["date"] >= pd.to_datetime("2013-03-29")) & (train["date"] < pd.to_datetime("2013-04-01")) # Thinking that the holiday will last for 3 days
during_easter = train[during_index].groupby("date").agg({"Sales": "mean"})

after_index = (train["date"] >= pd.to_datetime("2013-04-02")) & (train["date"] < pd.to_datetime("2013-05-01"))# Taking the one month after easter
after_easter = train[after_index].groupby("date").agg({"Sales": "mean"})
# Plotting the graph of Easter sales before, after and during

plt.figure(figsize=(12, 6))

sns.lineplot(x = before_easter.index, y = before_easter["Sales"], label='Before')
sns.lineplot(x = during_easter.index, y = during_easter["Sales"], label='During')
sns.lineplot(x = after_easter.index, y = after_easter["Sales"], label='After')

plt.title("Easter Sales 2013/2014", size=20)
plt.xticks(rotation=75, fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel(xlabel="Date", fontsize=16)
plt.ylabel(ylabel="Avg Sales", fontsize=16)
plt.savefig('../charts/easter_sales_analysis_13-14.jpg')
plt.show()

In [ ]:
# Creating the before after and during time stamp dataframe for 2014/15

before_index = (train["date"] >= pd.to_datetime("2015-03-03")) & (train["date"] < pd.to_datetime("2015-04-03")) # Taking the one month before easter
before_easter = train[before_index].groupby("date").agg({"Sales": "mean"})

during_index = (train["date"] >= pd.to_datetime("2015-04-03")) & (train["date"] < pd.to_datetime("2015-04-06")) # Thinking that the holiday will last for 6 days
during_easter = train[during_index].groupby("date").agg({"Sales": "mean"})

after_index = (train["date"] >= pd.to_datetime("2015-04-06")) & (train["date"] < pd.to_datetime("2015-05-06"))# Taking the one month after easter
after_easter = train[after_index].groupby("date").agg({"Sales": "mean"})
# Plotting the graph of Easter sales before, after and during

plt.figure(figsize=(12, 6))

sns.lineplot(x = before_easter.index, y = before_easter["Sales"], label='Before')
sns.lineplot(x = during_easter.index, y = during_easter["Sales"], label='During')
sns.lineplot(x = after_easter.index, y = after_easter["Sales"], label='After')

plt.title("Easter Sales 2014/2015", size=20)
plt.xticks(rotation=75, fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel(xlabel="Date", fontsize=16)
plt.ylabel(ylabel="Avg Sales", fontsize=16)
plt.savefig('../charts/easter_sales_analysis_14-15.jpg')
plt.show()

In [ ]:
# Pointing out the fields for correlation check

fields_to_check_correlation = ['Sales','Customers']
sample_check_for_corr = train[fields_to_check_correlation]
sample_check_for_corr.head()

In [ ]:
# Creating a correlation heatmap

corr = sample_check_for_corr.corr()
plt.figure(figsize=(10,5))
sns.heatmap(data=corr, annot = True)
plt.savefig('../charts/correlation_heatmap.jpg')
plt.show()